# Stage 1: Generate Stock Universe

- Gather stocks of interest
- Gather stocks from specific criteria (SP500 top 50...)
- Gather stocks from specific portfolio account
- Assemble stock universe 
- Use stock sentiment to select stocks
- Gather price histories

In [1]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '../tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import utils
importlib.reload(utils)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.8.10
Pandas version: 0.25.3


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

## Authentication Tokens

To get data from Ameritrade you will need to obtains a short time use token (there is a re-use token, but I have not coded it yet.) You only need to do this if you
are going to use an existing Ameritrade account to define an initial set of stocks to analyze.

To obtain a token, you will need to have a Chrome driver located somewhere on your system. This will allow the module to use your credentials to obtain an authentication token.

For security reasons, I sugges using environment variables to store your credential information. If you store them in property files, or just code them into your notebook, you risk sharing the information with others if you use GitHub or some other SCCS. This also makes it easier to have them availabe from project to project in your development environment

<span style="color:blue">Note: *Account numbers are masked for security purposes.*</span>

In [2]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

# For Chromedriver
from pathlib import Path
chrome_executabel_path = str(Path.home()) + r'\Anaconda Projects\chromedriver\chromedriver'

# Make sure we have a data directory
Path('./data').mkdir(parents=True, exist_ok=True) 

# Which account are we interested in
masked_account_number = '#---9216'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories.csv'

## Stock Universe

Here we setup the univers. This needs some work. The long term goal is to use a pipeline process to help select stock that are in the top 500 or something similare.

For now we will use stocks from the portfolio, but stocks of interest (high news items), a list of well known stocks (this also has been augmented with some stocks that made Ameritrade's top 10 movers for a couple of days. This Ameritrade funciton has not been coded yet, but should be add down the line to automate pulling these tickers.

In [3]:
snp_500_df = utils.get_snp500()
snp_500_symbols = snp_500_df.index.to_list()
quote_dfs = []
for i in range(0, 500, 100):
    quote_dfs.append(amc.AmeritradeRest(username, password, client_id).get_quotes(snp_500_symbols[i:i+100]))
    
snp_500_quotes_df = pd.concat(quote_dfs, axis=0)
snp_500_quotes_df.describe()

snp_500_tickers = set(snp_500_df.index.to_list())
print(len(snp_500_tickers))

505


In [4]:
dow_tikers = set(utils.get_dow().index.to_list())
print(len(dow_tikers))

30


## Get sentiment data from Finvis

In [5]:
stock_universe = set.union(snp_500_tickers, dow_tikers)
print(len(stock_universe))

505


In [6]:
parsed_and_scored_news = utils.get_finvis_stock_sentiment(snp_500_symbols).sort_values(by='date')
parsed_and_scored_news

News Tables: 100%|████████████████████████████████████████████████████| 502/502 [00:02<00:00, 215.05News Table Items/s]


,ticker,date,time,headline,neg,neu,pos,compound
29104,L,2019-10-28,09:00AM,Loews Corp. to Host Earnings Call,0.000,1.000,0.000,0.0000
29106,L,2019-10-28,06:00AM,Loews Corporation Reports Net Income Of $72 Mi...,0.000,1.000,0.000,0.0000
29103,L,2019-10-28,12:19PM,"Loews (L) Q3 Earnings Miss Estimates, Revenues...",0.167,0.833,0.000,-0.1531
29102,L,2019-10-28,03:26PM,Global Markets Advance Monday,0.000,1.000,0.000,0.0000
29105,L,2019-10-28,06:11AM,Loews reports slump in third-quarter profit as...,0.000,0.775,0.225,0.4404
...,...,...,...,...,...,...,...,...
49120,WYNN,2021-10-08,08:00AM,Wynn Resorts Recognized As One Of The Top Plac...,0.000,0.870,0.130,0.2023
21307,GILD,2021-10-08,05:36AM,10 Best Big Pharma Stocks to Buy Now,0.000,0.625,0.375,0.6369
22107,HAS,2021-10-08,09:00AM,Hasbro Hires Carrie Ratner to Lead Global Corp...,0.000,1.000,0.000,0.0000
19307,FISV,2021-10-08,08:04AM,Top 10 Stocks to Buy According to Jeff Ubbens ...,0.000,0.847,0.153,0.2023


In [7]:
# Group by date and ticker columns from scored_news and calculate the mean
mean_scores = parsed_and_scored_news.groupby(['ticker','date']).mean().fillna(0)
# Unstack the column ticker
mean_scores = mean_scores.unstack()
# Get the cross-section of compound in the 'columns' axis
mean_scores = mean_scores.xs('compound', axis="columns").transpose().fillna(0)
# Get cusmum score of each stock
cum_scores = mean_scores[-40:].cumsum(axis=0)
current_scores = cum_scores.iloc[-1]
mean_score = current_scores.mean()
stdv_score = current_scores.std()
cutoff = mean_score - stdv_score

print(mean_score, stdv_score, cutoff)

2.2812703959616822 1.8077288249541983 0.47354157100748395


In [8]:
stock_universe = current_scores.where(current_scores > cutoff).dropna().index.to_list()

# Price History data

One you have a set of investments you want to work with, you will need to pull some historical data for them.

We will obtain 5 years of price histories. In the end this will provide us with 2 years of factor data since some of the factors are based on 1 year returns.

In [9]:
number_of_years = 5
price_histories = amc.AmeritradeRest(username, password, client_id).get_price_histories(stock_universe, datetime.today().strftime('%Y-%m-%d'), num_periods=number_of_years)
utils.save_price_histories(price_histories, price_histories_file_name)

Tickers:   0%|          | 0/436 [00:00<?, ?Price Histories/s]

In [10]:
price_histories.head()

,open,high,low,close,volume,ticker,date
0,47.58,48.15,47.55,47.69,1544958,A,2016-10-10
457743,38.93,39.20,38.21,39.01,19020705,T,2016-10-10
89910,61.48,61.91,61.42,61.79,1070121,CERN,2016-10-10
456485,48.10,48.51,48.10,48.21,2639966,SYY,2016-10-10
91168,24.16,24.35,23.80,23.86,3346556,CF,2016-10-10


In [11]:
price_histories = utils.read_price_histories(price_histories_file_name)
close = utils.get_close_values(price_histories)
close.tail()

ticker,A,AAL,ABBV,ABC,ABT,ACN,ADBE,ADI,ADM,ADP,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2021-10-01 00:00:00+00:00,155.99,21.65,109.09,119.81,117.21,324.86,577.47,167.37,60.80,202.00,...,35.27,87.21,62.49,60.93,59.07,121.30,123.64,149.47,510.98,196.19
2021-10-04 00:00:00+00:00,152.36,21.39,108.73,118.33,115.53,319.49,558.49,164.26,61.16,199.97,...,35.72,83.56,63.90,61.72,57.63,119.30,123.04,146.73,502.32,193.69
2021-10-05 00:00:00+00:00,153.03,21.47,109.51,118.61,115.88,325.12,566.70,165.37,62.28,203.76,...,35.73,85.05,63.30,61.62,57.58,122.47,123.60,146.01,506.89,196.46
2021-10-06 00:00:00+00:00,153.65,20.54,109.32,116.90,116.49,325.19,570.31,166.05,62.30,204.99,...,35.82,84.25,64.33,60.49,57.46,122.25,123.66,146.34,506.39,195.78
2021-10-07 00:00:00+00:00,155.32,20.17,110.87,118.57,117.91,326.25,578.96,166.98,63.02,207.57,...,35.98,85.49,63.97,60.66,58.00,122.36,125.22,147.74,504.41,199.01
